# Feature attribution

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2022"

## Contents

1. [Overview](#Overview)
1. [InputXGradients](#InputXGradients)
1. [Selectivity examples](#Selectivity-examples)
1. [Simple feed-forward classifier example](#Simple-feed-forward-classifier-example)
1. [Bag-of-words classifier for the SST](#Bag-of-words-classifier-for-the-SST)
1. [BERT example](#BERT-example)

## Overview

This notebook is an experimental extension of the CS224u course code. It focuses on the [Integrated Gradients](https://arxiv.org/abs/1703.01365) method for feature attribution, with comparisons to the "inputs $\times$ gradients" method. To run the notebook, first install [the Captum library](https://captum.ai/):

In [2]:
!pip install captum

This is not currently a required installation (but it will be in future years).

## InputXGradients

For both implementations, the `forward` method of `model` is used. `X` is an (m x n) tensor of attributions. Use `targets=None` for models with scalar outputs, else supply a LongTensor giving a label for each example.

In [1]:
import torch

def grad_x_input(model, X, targets=None):
    """Implementation using PyTorch directly."""
    X.requires_grad = True
    y = model(X)
    y = y if targets is None else y[list(range(len(y))), targets]
    (grads, ) = torch.autograd.grad(y.unbind(), X)
    return grads * X

In [2]:
from captum.attr import InputXGradient

def captum_grad_x_input(model, X, target):
    """Captum-based implementation."""
    X.requires_grad = True
    amod = InputXGradient(model)
    return amod.attribute(X, target=target)

## Selectivity examples

In [3]:
import numpy as np
import torch
import torch.nn as nn
from captum.attr import IntegratedGradients
from captum.attr import InputXGradient

In [4]:
class SelectivityAssessor(nn.Module):
    """Model used by Sundararajan et al, section 2.1 to show that
    input * gradients violates their selectivity axiom.
    """
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()

    def forward(self, X):
        return 1.0 - self.relu(1.0 - X)

In [6]:
sel_mod = SelectivityAssessor()

Simple inputs with just one feature:

In [7]:
X_sel = torch.FloatTensor([[0.0], [2.0]])

The outputs for our two examples differ:

In [8]:
sel_mod(X_sel)

tensor([[0.],
        [1.]])

However, `InputXGradient` assigns the same importance to the feature across the two examples, violating selectivity:

In [9]:
captum_grad_x_input(sel_mod, X_sel, target=None)

tensor([[0.],
        [-0.]], grad_fn=<MulBackward0>)

Integrated gradients addresses the problem by averaging gradients across all interpolated representations between the baseline and the actual input:

In [10]:
ig_sel = IntegratedGradients(sel_mod)

In [11]:
sel_baseline = torch.FloatTensor([[0.0]])

In [12]:
ig_sel.attribute(X_sel, sel_baseline)

tensor([[0.],
        [1.]], dtype=torch.float64, grad_fn=<MulBackward0>)

A toy implementation to help bring out what is happening:

In [14]:
def ig_reference_implementation(model, x, base, m=50):
    vals = []
    for k in range(m):
        # Interpolated representation:
        xx = (base + (k/m)) * (x - base)
        # Gradient for the interpolated example:
        xx.requires_grad = True
        y = model(xx)
        (grads, ) = torch.autograd.grad(y.unbind(), xx)
        vals.append(grads)
    return (1 / m) * torch.cat(vals).sum(axis=0) * (x - base)

In [18]:
ig_reference_implementation(sel_mod, torch.FloatTensor([[20.0]]), sel_baseline)

tensor([[1.2000]])

## Simple feed-forward classifier example

In [19]:
from captum.attr import IntegratedGradients
from sklearn.datasets import make_classification
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

In [20]:
X_cls, y_cls = make_classification(
    n_samples=5000,
    n_classes=3,
    n_features=5,
    n_informative=3,
    n_redundant=0,
    random_state=42)

The classification problem has two uninformative features:

In [21]:
mutual_info_classif(X_cls, y_cls)

array([0.20138107, 0.02833358, 0.11584416, 0.        , 0.        ])

In [22]:
X_cls.shape

(5000, 5)

In [23]:
y_cls.shape

(5000,)

In [24]:
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X_cls, y_cls)

In [25]:
classifier = TorchShallowNeuralClassifier()

In [26]:
_ = classifier.fit(X_cls_train, y_cls_train)

Stopping after epoch 399. Training loss did not improve more than tol=1e-05. Final error is 1.4130553603172302.

In [27]:
cls_preds = classifier.predict(X_cls_test)

In [28]:
accuracy_score(y_cls_test, cls_preds)

0.8648

In [29]:
classifier_ig = IntegratedGradients(classifier.model)

In [30]:
classifier_baseline = torch.zeros(1, X_cls_train.shape[1])

Integrated gradients with respect to the actual labels:

In [31]:
classifier_attrs = classifier_ig.attribute(
    torch.FloatTensor(X_cls_test),
    classifier_baseline,
    target=torch.LongTensor(y_cls_test))

Average attribution is low for the two uninformative features:

In [32]:
classifier_attrs.mean(axis=0)

tensor([ 1.0526,  0.6108,  0.4918, -0.0254, -0.0277], dtype=torch.float64)

## Bag-of-words classifier for the SST

In [33]:
from collections import Counter
from captum.attr import IntegratedGradients
from nltk.corpus import stopwords
from operator import itemgetter
import os
from sklearn.metrics import classification_report
import torch
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import sst

In [34]:
SST_HOME = os.path.join("data", "sentiment")

Bag-of-word featurization with stopword removal to make this a little easier to study:

In [36]:
import nltk
nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

def phi(text):
    return Counter([w for w in text.lower().split() if w not in stopwords])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xianbing/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
def fit_mlp(X, y):
    mod = TorchShallowNeuralClassifier(early_stopping=True)
    mod.fit(X, y)
    return mod

In [38]:
experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    phi,
    fit_mlp,
    sst.dev_reader(SST_HOME))

Stopping after epoch 41. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.5985297821462154

              precision    recall  f1-score   support

    negative      0.633     0.673     0.652       428
     neutral      0.293     0.157     0.205       229
    positive      0.639     0.752     0.691       444

    accuracy                          0.598      1101
   macro avg      0.521     0.527     0.516      1101
weighted avg      0.564     0.598     0.575      1101



Trained model:

In [39]:
sst_classifier = experiment['model']

Captum needs to have labels as indices rather than strings:

In [40]:
sst_classifier.classes_

['negative', 'neutral', 'positive']

In [41]:
y_sst_test = [sst_classifier.classes_.index(label)
              for label in experiment['assess_datasets'][0]['y']]

sst_preds = [sst_classifier.classes_.index(label)
             for label in experiment['predictions'][0]]

Our featurized test set:

In [42]:
X_sst_test = experiment['assess_datasets'][0]['X']

Feature names to help with analyses:

In [43]:
fnames = experiment['train_dataset']['vectorizer'].get_feature_names()

/Users/xianbing/opt/anaconda3/envs/nlu/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
fnames

['!',
 '!?',
 '#',
 '$',
 '&',
 "'",
 "''",
 "'30s",
 "'40s",
 "'50s",
 "'53",
 "'60s",
 "'70s",
 "'80s",
 "'90s",
 "'d",
 "'em",
 "'ll",
 "'m",
 "'n",
 "'n'",
 "'re",
 "'s",
 "'til",
 "'ve",
 '+',
 ',',
 '-',
 '--',
 '-lrb-',
 '-rrb-',
 '.',
 '...',
 '1',
 '1.2',
 '1.8',
 '10',
 '10,000',
 '10-course',
 '10-year',
 '10-year-old',
 '100',
 '100-minute',
 '100-year',
 '101',
 '102-minute',
 '103-minute',
 '104',
 '105',
 '10th',
 '10th-grade',
 '11',
 '110',
 '112-minute',
 '12',
 '12-year-old',
 '120',
 '127',
 '129-minute',
 '12th',
 '13',
 '13th',
 '14-year-old',
 '140',
 '146',
 '15',
 '15-year',
 '15th',
 '163',
 '168-minute',
 '170',
 '1790',
 '18',
 '18-year-old',
 '1899',
 '19',
 '1915',
 '1920',
 '1930s',
 '1933',
 '1937',
 '1938',
 '1940s',
 '1950',
 '1950s',
 '1952',
 '1953',
 '1957',
 '1958',
 '1959',
 '1960',
 '1960s',
 '1962',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1975',
 '1979',
 '1980',
 '1980s',
 '1984',
 '1986',
 '1987',
 '1989',
 '1990',
 '1991',
 '1992',
 '

Integrated gradients:

In [44]:
sst_ig = IntegratedGradients(sst_classifier.model)

All-0s baseline:

In [45]:
sst_baseline = torch.zeros(1, experiment['train_dataset']['X'].shape[1])

Attributions with respect to the model's predictions:

In [46]:
sst_attrs = sst_ig.attribute(
    torch.FloatTensor(X_sst_test),
    sst_baseline,
    target=torch.LongTensor(sst_preds))

Helper functions for error analysis:

In [48]:
def error_analysis(gold=1, predicted=2):
    err_ind = [i for i, (g, p) in enumerate(zip(y_sst_test, sst_preds))
               if g == gold and p == predicted]
    attr_lookup = create_attr_lookup(sst_attrs[err_ind])
    return attr_lookup, err_ind

def create_attr_lookup(attrs):
    mu = attrs.mean(axis=0).detach().numpy()
    return sorted(zip(fnames, mu), key=itemgetter(1), reverse=True)

In [49]:
sst_attrs_lookup, sst_err_ind = error_analysis(gold=1, predicted=2)

In [50]:
sst_attrs_lookup[: 5]

[('.', 0.11495877629674077),
 ('fun', 0.06644722713237726),
 ('film', 0.04247883257980162),
 ('solid', 0.04107643978858098),
 ('makes', 0.03878941709677621)]

Error analysis for a specific example:

In [51]:
ex_ind = sst_err_ind[0]

In [52]:
experiment['assess_datasets'][0]['raw_examples'][ex_ind]

'No one goes unindicted here , which is probably for the best .'

In [53]:
ex_attr_lookup = create_attr_lookup(sst_attrs[ex_ind:ex_ind+1])

In [54]:
[(f, a) for f, a in ex_attr_lookup if a != 0]

[('best', 0.9777032930161551),
 ('probably', 0.23590500013472715),
 ('.', 0.11888355179360353),
 (',', 0.03269009976933634),
 ('one', 0.00562289517486443),
 ('goes', -0.06158406715880671)]

## BERT example

In [48]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

In [49]:
hf_weights_name = 'cardiffnlp/twitter-roberta-base-sentiment'

In [50]:
hf_tokenizer = AutoTokenizer.from_pretrained(hf_weights_name)

In [51]:
hf_model = AutoModelForSequenceClassification.from_pretrained(hf_weights_name)

In [52]:
def hf_predict_one_proba(text):
    input_ids = hf_tokenizer.encode(
        text, add_special_tokens=True, return_tensors='pt')
    hf_model.eval()
    with torch.no_grad():
        logits = hf_model(input_ids)[0]
        preds = F.softmax(logits, dim=1)
    hf_model.train()
    return preds.squeeze(0)

In [53]:
def hf_ig_encodings(text):
    pad_id = hf_tokenizer.pad_token_id
    cls_id = hf_tokenizer.cls_token_id
    sep_id = hf_tokenizer.sep_token_id
    input_ids = hf_tokenizer.encode(text, add_special_tokens=False)
    base_ids = [pad_id] * len(input_ids)
    input_ids = [cls_id] + input_ids + [sep_id]
    base_ids = [cls_id] + base_ids + [sep_id]
    return torch.LongTensor([input_ids]), torch.LongTensor([base_ids])

In [54]:
def hf_ig_analyses(text2class):
    data = []
    for text, true_class in text2class.items():
        score_vis = hf_ig_analysis_one(text, true_class)
        data.append(score_vis)
    viz.visualize_text(data)


def hf_ig_analysis_one(text, true_class):
    # Option to look at different layers:
    # layer = model.roberta.encoder.layer[0]
    # layer = model.roberta.embeddings.word_embeddings
    layer = hf_model.roberta.embeddings

    def ig_forward(inputs):
        return hf_model(inputs).logits

    ig = LayerIntegratedGradients(ig_forward, layer)

    input_ids, base_ids = hf_ig_encodings(text)

    attrs, delta = ig.attribute(
        input_ids,
        base_ids,
        target=true_class,
        return_convergence_delta=True)

    # Summarize and z-score normalize the attributions
    # for each representation in `layer`:
    scores = attrs.sum(dim=-1).squeeze(0)
    scores = (scores - scores.mean()) / scores.norm()

    # Intuitive tokens to help with analysis:
    raw_input = hf_tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])
    # RoBERTa-specific clean-up:
    raw_input = [x.strip("Ġ") for x in raw_input]

    # Predictions for comparisons:
    pred_probs = hf_predict_one_proba(text)
    pred_class = pred_probs.argmax()

    score_vis = viz.VisualizationDataRecord(
        word_attributions=scores,
        pred_prob=pred_probs.max(),
        pred_class=pred_class,
        true_class=true_class,
        attr_class=None,
        attr_score=attrs.sum(),
        raw_input_ids=raw_input,
        convergence_score=delta)

    return score_vis

In [55]:
score_vis = hf_ig_analyses({
    "They said it would be great, and they were right.": 2,
    "They said it would be great, and they were wrong.": 0,
    "They were right to say it would be great.": 2,
    "They were wrong to say it would be great.": 0,
    "They said it would be stellar, and they were correct.": 2,
    "They said it would be stellar, and they were incorrect.": 0})

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (0.82),None,1.98,"#s They said it would be great , and they were right . #/s"
0,0 (0.50),None,0.07,"#s They said it would be great , and they were wrong . #/s"
2,2 (0.76),None,2.39,#s They were right to say it would be great . #/s
0,0 (0.62),None,3.46,#s They were wrong to say it would be great . #/s
2,2 (0.77),None,1.78,"#s They said it would be stellar , and they were correct . #/s"
0,1 (0.47),None,1.17,"#s They said it would be stellar , and they were incorrect . #/s"
